# Testing and Debugging a New Forward Pass Approach

This document explores a novel forward pass implementation that replaces the traditional 3D tensor input with boolean masking. Instead, this approach uses a 2D tensor as input combined with a 1D tensor that assigns group labels to each peptide, offering a more efficient and flexible alternative for batch processing.

Imports

In [27]:
import torch
import torch.nn as nn

Mock input tensors

In [28]:
torch.manual_seed(0)

X_tensor = torch.randn(5, 180, requires_grad=True)
group = torch.tensor([0, 0, 0, 1, 1], dtype=torch.long)

print(X_tensor.shape)
print(group)

torch.Size([5, 180])
tensor([0, 0, 0, 1, 1])


Forward pass 

In [29]:
in_layer = nn.Linear(180, 66)
out_layer = nn.Linear(66, 1)
activation = nn.Tanh()

In [30]:
z = in_layer(X_tensor)
z = activation(z)
z = out_layer(z).squeeze(-1)

print(z)

tensor([-0.2603,  0.6663,  0.3373,  0.5380,  0.1768],
       grad_fn=<SqueezeBackward1>)


Selection of the max loggits in groups

In [31]:
batch_size = int(group.max().item()) + 1
print(batch_size)

2


In [32]:
z_max = torch.full((batch_size,), float("-inf"), device=z.device, dtype=z.dtype) 
print(z_max)

tensor([-inf, -inf])


In [33]:
z_max = torch.scatter_reduce(z_max, 0, group, z, reduce="amax", include_self=True)
print(z_max)

tensor([0.6663, 0.5380], grad_fn=<ScatterReduceBackward0>)
